In [1]:
%load_ext autotime 

time: 0 ns (started: 2023-07-29 17:17:35 +05:30)


In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import (
    activations,
    callbacks,
    preprocessing,
    layers,
    losses,
    metrics,
    models,
    optimizers,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import (
    classification_report,
    RocCurveDisplay,
    ConfusionMatrixDisplay,
)

np.set_printoptions(precision=2, suppress=True)

import warnings

warnings.filterwarnings("ignore", "all")


time: 0 ns (started: 2023-07-29 17:18:54 +05:30)


In [3]:
data = pd.read_csv("../data/Merged/reviews_cleaned.csv")

time: 156 ms (started: 2023-07-29 17:17:42 +05:30)


In [4]:
MAX_FEATURES = 30_000
MAX_LEN = 200

time: 16 ms (started: 2023-07-29 17:17:42 +05:30)


In [5]:
def build_vectorizer() -> layers.TextVectorization:
    vec = layers.TextVectorization(
        max_tokens=MAX_FEATURES,
        output_sequence_length=MAX_LEN,
        pad_to_max_tokens=True,
        encoding="int",
    )

    return vec


time: 0 ns (started: 2023-07-29 17:17:42 +05:30)


In [6]:
vectorizer = build_vectorizer()
vectorizer.adapt(data["review_text_cleaned"])

time: 1.55 s (started: 2023-07-29 17:17:45 +05:30)


In [7]:
def build_model(vectorizer: layers.TextVectorization) -> models.Model:
    text_input = keras.Input(shape=(1,), dtype=tf.string, name="text")
    X = vectorizer(text_input)
    X = layers.Embedding(MAX_FEATURES, 128)(X)

    X = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(X)
    X = layers.Bidirectional(layers.LSTM(64))(X)
    outputs = layers.Dense(1, activation="sigmoid")(X)

    model = keras.Model(inputs=text_input, outputs=outputs)
    model.compile(
        loss=losses.categorical_crossentropy,
        optimizer=optimizers.RMSprop(learning_rate=1e-3),
        metrics=[metrics.AUC(name="auc"), metrics.CategoricalAccuracy(name="acc")],
    )

    return model


model = build_model(vectorizer)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text (InputLayer)           [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 200)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 200, 128)          3840000   
                                                                 
 bidirectional (Bidirection  (None, 200, 128)          98816     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                             

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    data["review_text_cleaned"], data["review_type"], train_size=0.7
)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, train_size=0.5)

enc = OneHotEncoder()
enc.fit(y_train.to_numpy().reshape(1, -1))

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)
y_val = enc.transform(y_val)


ValueError: X does not contain any features, but OneHotEncoder is expecting 21000 features

time: 296 ms (started: 2023-07-29 17:23:43 +05:30)


In [9]:
callbacks = [
    callbacks.EarlyStopping(
        monitor="val_auc", mode="max", patience=10, restore_best_weights=True
    ),
    callbacks.ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.7, patience=4),
]

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=60,
    batch_size=64,
    callbacks=callbacks,
)


Epoch 1/60


c:\wrkspc\projects\xebia_internship\.venv\Lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


UnimplementedError: Graph execution error:

Detected at node 'categorical_crossentropy/Cast' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "c:\wrkspc\projects\xebia_internship\.venv\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\wrkspc\projects\xebia_internship\.venv\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\wrkspc\projects\xebia_internship\.venv\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "c:\wrkspc\projects\xebia_internship\.venv\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Python311\Lib\asyncio\base_events.py", line 607, in run_forever
      self._run_once()
    File "C:\Python311\Lib\asyncio\base_events.py", line 1922, in

time: 4.58 s (started: 2023-07-29 17:17:57 +05:30)


In [ ]:
def plot_hist(hist, metric='auc'):
    plt.plot(hist.history[metric])
    plt.plot(hist.history["val_" + metric])
    plt.title(f"model performance")
    plt.ylabel("area_under_curve")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()
    return